# YOLOv11-seg LGG MRI – FINAL WORKING VERSION


In [ ]:
# STEP 1: Fix everything in one go (NO libGL error, NO torch conflict)
!pip install -q --force-reinstall --no-cache-dir \
    torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 \
    --index-url https://download.pytorch.org/whl/cu118

!pip install -q --force-reinstall --no-cache-dir \
    ultralytics kaggle tqdm

# OpenCV headless = no libGL error EVER
!pip install -q --force-reinstall --no-cache-dir opencv-python-headless==4.10.0.84

import torch
from IPython.display import FileLink, display, clear_output
import os, cv2, numpy as np, random
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO

print("All packages fixed & installed!")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not found'}")

In [ ]:
# STEP 2: Upload kaggle.json (drag & drop)
print("Please DRAG & DROP kaggle.json into the left file panel")
print("Then run this cell again")

clear_output(wait=True)

if not os.path.exists("kaggle.json"):
    print("kaggle.json not found! Please upload it.")
else:
    !mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    print("kaggle.json ready!")

In [ ]:
# STEP 3: Download dataset
!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation -p data --unzip
print("Dataset downloaded!")

In [ ]:
# STEP 4: Convert to YOLO format
raw = Path("data/lgg-mri-segmentation")
yolo = Path("yolo_dataset")
for p in ["images/train","images/val","labels/train","labels/val"]:
    (yolo/p).mkdir(parents=True, exist_ok=True)

pairs = []
for patient in raw.iterdir():
    if patient.is_dir():
        for img in patient.glob("*[0-9].tif"):
            mask = patient / f"{img.stem}_mask.tif"
            if mask.exists():
                pairs.append((img, mask))

random.seed(42)
random.shuffle(pairs)
split = int(0.8 * len(pairs))

def convert(img_path, mask_path, split_folder):
    img = cv2.imread(str(img_path))
    mask = cv2.imread(str(mask_path), 0)
    h, w = img.shape[:2]
    name = f"{img_path.parent.name}_{img_path.name}"
    cv2.imwrite(str(yolo/f"images/{split_folder}/{name}"), img)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    segs = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 30:
            c = cnt.flatten().astype(float)
            c[0::2] /= w
            c[1::2] /= h
            segs.append(f"0 {' '.join(map(str, c))}")
    if segs:
        with open(yolo/f"labels/{split_folder}/{name.replace('.tif','.txt')}", "w") as f:
            f.write("\n".join(segs))

for p in tqdm(pairs[:split], desc="Train"): convert(p[0], p[1], "train")
for p in tqdm(pairs[split:], desc="Val")  : convert(p[0], p[1], "val")

with open("data.yaml", "w") as f:
    f.write("""path: yolo_dataset
train: images/train
val: images/val
nc: 1
names: ['tumor']""")

print("YOLO dataset ready!")

In [ ]:
# STEP 5: ULTRA FAST GPU TRAINING (25–35 min)
model = YOLO("yolov11n-seg.pt")

model.train(
    data="data.yaml",
    epochs=80,
    imgsz=224,
    batch=16,
    device=0,
    project="runs",
    name="final",
    exist_ok=True,
    patience=20,
    amp=True,
    workers=4
)
print("Training finished!")

In [ ]:
# STEP 6: Create demo video
best = YOLO("runs/final/weights/best.pt")
imgs = sorted(list(Path("yolo_dataset/images/val").glob("*.tif")))[:30]

frames = []
for p in tqdm(imgs, desc="Video"):
    img = cv2.imread(str(p))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = best(p, conf=0.25, verbose=False)[0]
    overlay = img_rgb.copy()
    if res.masks is not None:
        for mask, box in zip(res.masks.data.cpu(), res.boxes.data.cpu()):
            m = cv2.resize(mask.numpy(), overlay.shape[:2][::-1]) > 0.5
            overlay[m] = overlay[m] * 0.6 + np.array([0,191,255]) * 0.4
            cv2.putText(overlay, f"Tumor {box[4]:.2f}", (50,80), 1, 2, (255,255,255), 4)
    cv2.putText(overlay, "YOLOv11-seg LGG MRI | Omid Sakaki", (20,img_rgb.shape[0]-30), 1, 0.9, (255,255,255), 2)
    frames.append(overlay)

out = cv2.VideoWriter("Demo.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 6, (img_rgb.shape[1], img_rgb.shape[0]))
for f in frames: out.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
out.release()

display(FileLink("Demo.mp4"))
print("ALL DONE! Click the link to download your video")